In [50]:
import pandas as pd
import json
import random

In [51]:
data = pd.read_excel('climbingdata.xlsx', sheet_name = 3, usecols=[0,1,2,4,5,6,7,8,9,10,11,12,13,14])
data.head()

,Climb,Area,Climber,FrenchGrade,Style,Gender,Year,Grade,OriginalArea,SuggestedGrade,AscentNumber,Notes,SuggestedNumberGrade,AveragedNumberGrade
0,Kemena,"12590 Almenara, Spain",Rubén Díaz Torres,8C,Boulder Problem,M,2013,V15,Almenara,8C,1,NaN,15.0,15
1,Kemena,"12590 Almenara, Spain",Ignacio Sánchez González,8C,Boulder Problem,M,2013,V15,Almenara,8B+,2,NaN,14.0,15
2,Hurricane,5505 Mühlbach am Hochkönig,Bernhard Schwaiger,8C+,Boulder Problem,M,2009,V16,Birgkar,8C/8C+,1,8C/(8C+),15.5,16
3,L'Anarchiste,"83603 Saint-Anne, France",Philip Moser,8C,Boulder Problem,M,2003,V15,St. Anne,8C,1,NaN,15.0,15
4,L'Anarchiste,"83603 Saint-Anne, France",Frédéric Tuscan,8C,Boulder Problem,M,2004,V15,St. Anne,8C,2,NaN,15.0,15


In [52]:
sport = data[(data.Style == "Sport Route") & (data.Grade != "5.14d")]
boulder = data[(data.Style == "Boulder Problem") & (data.Grade != "V14")]
boulder_small = boulder[boulder["Grade"] != "V15"]

In [53]:
x = sport.groupby(by ='Climber').apply(lambda g: g.to_dict(orient='records'))
y = boulder.groupby(by ='Climber').apply(lambda g: g.to_dict(orient='records'))
y2 = boulder_small.groupby(by ='Climber').apply(lambda g: g.to_dict(orient='records'))

In [54]:
with open('boulder_small.json', 'w', encoding='utf8') as outfile:  
    json.dump(dict(y2), outfile, ensure_ascii=False)
with open('boulder.json', 'w', encoding='utf8') as outfile:  
    json.dump(dict(y), outfile, ensure_ascii=False)
with open('sport.json', 'w', encoding='utf8') as outfile:  
    json.dump(dict(x), outfile, ensure_ascii=False)

In [64]:
forceData = {'nodes': [], 'links': []}
climbers = []
climbs = []
nodeIndices = {}
n = 0
for row in data.iterrows():
    if (row[1].FrenchGrade != '8B+') & (row[1].FrenchGrade != '9a'):
        if row[1].Climber not in climbers:
            node = {
                'id': row[1].Climber,
                'color': random.choice(['#8dd3c7', '#bebada', '#fb8072', '#80b1d3', '#fdb462', '#b3de69', '#fccde5']),
                'grade': 0,
                'style': 0,
                '8C': 0,
                '8C+': 0,
                '9A': 0,
                '9a+': 0,
                '9b': 0,
                '9b+': 0,
                '9c': 0
            }
            node[row[1].FrenchGrade] += 1
            forceData['nodes'].append(node)
            nodeIndices[row[1].Climber] = n
            n += 1
            climbers.append(row[1].Climber)
        else:
            forceData['nodes'][nodeIndices[row[1].Climber]][row[1].FrenchGrade] += 1
        if row[1].Climb not in climbs:
            node = {
                'id': row[1].Climb + ' ' + row[1].Grade,
                'color': '#A9A9A9',
                'grade': row[1].FrenchGrade,
                'style': row[1].Style,
                '8C': 0,
                '8C+': 0,
                '9A': 0,
                '9a+': 0,
                '9b': 0,
                '9b+': 0,
                '9c': 0
            }
            forceData['nodes'].append(node)
            nodeIndices[row[1].Climb] = n
            n += 1
            climbs.append(row[1].Climb)
        if row[1].AscentNumber == 1:
            firstAscent = True
        else:
            firstAscent = False
        link = {
            'source': row[1].Climber,
            'target': row[1].Climb + ' ' + row[1].Grade,
            'color': forceData['nodes'][nodeIndices[row[1].Climber]]['color'],
            'grade': row[1].FrenchGrade,
            'style': row[1].Style,
            'firstAscent': firstAscent,
            'value': 8
        }  
        forceData['links'].append(link)

In [65]:
with open('forceData.json', 'w', encoding='utf8') as outfile:  
    json.dump(forceData, outfile, ensure_ascii=False, indent=2)